# Turn the Review App logs into an Evaluation Set

The Review application captures your user feedbacks.

This feedback is saved under 2 tables within your schema.

In this notebook, we will show you how to extract the logs from the Review App into an Evaluation Set.  It is important to review each row and ensure the data quality is high e.g., the question is logical and the response makes sense.

1. Requests with a 👍 :
    - `request`: As entered by the user
    - `expected_response`: If the user edited the response, that is used, otherwise, the model's generated response.
2. Requests with a 👎 :
    - `request`: As entered by the user
    - `expected_response`: If the user edited the response, that is used, otherwise, null.
3. Requests without any feedback
    - `request`: As entered by the user

Across all types of requests, if the user 👍 a chunk from the `retrieved_context`, the `doc_uri` of that chunk is included in `expected_retrieved_context` for the question.

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=1444828305810485&notebook=%2F03-advanced-app%2F03-Offline-Evaluation&demo_name=llm-rag-chatbot&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fllm-rag-chatbot%2F03-advanced-app%2F03-Offline-Evaluation&version=1">

In [0]:
%pip install --quiet -U databricks-agents mlflow mlflow-skinny databricks-sdk==0.23.0 transformers==4.41.1 pypdf==4.1.0 langchain==0.2.1 
dbutils.library.restartPython()


## 1.1/ Extracting the logs 


*Note: for now, this part requires a few SQL queries that we provide in this notebook to properly format the review app into training dataset.*

*We'll update this notebook soon with an simpler version - stay tuned!*

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=1444828305810485&notebook=%2F03-advanced-app%2F03-Offline-Evaluation&demo_name=llm-rag-chatbot&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fllm-rag-chatbot%2F03-advanced-app%2F03-Offline-Evaluation&version=1">

In [0]:
%run ../_resources/00-init-advanced $reset_all_data=false

In [0]:
from databricks import agents
MODEL_NAME = "rag_demo_advanced"
MODEL_NAME_FQN = f"{catalog}.{db}.{MODEL_NAME}"
browser_url = mlflow.utils.databricks_utils.get_browser_hostname()

# # Get the name of the Inference Tables where logs are stored
active_deployments = agents.list_deployments()
active_deployment = next((item for item in active_deployments if item.model_name == MODEL_NAME_FQN), None)

# 1.2/ Our eval dataset is now ready! 

The review app makes it easy to build & create your evaluation dataset. 

*Note: the eval app logs may take some time to be available to you. If the dataset is empty, wait a bit.*

To simplify the demo and make sure you don't have to craft your own eval dataset, we saved a ready-to-use eval dataset already pre-generated for you. We'll use this one for the demo instead.

In [0]:
eval_dataset = spark.table(f"{catalog}.{db}.eval_dataset")
display(eval_dataset)

## Load the correct Python environment for the model


In [0]:
#Retrieve the model we want to eval
model = get_latest_model(MODEL_NAME_FQN)
pip_requirements = mlflow.pyfunc.get_model_dependencies(f"runs:/{model.run_id}/chain")

In [0]:
%pip install -r $pip_requirements

## Run our evaluation from the dataset!

In [0]:
with mlflow.start_run(run_name="eval_dataset_advanced"):
    # Evaluate the logged model
    eval_results = mlflow.evaluate(
        data=eval_dataset,
        model=f'runs:/{model.run_id}/chain',
        model_type="databricks-agent",
    )

You can open MLFlow and review the eval metrics, and also compare it to previous eval runs!

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/mlflow-eval.gif?raw=true" width="1200px"> 

### This is looking good, let's tag our model as production ready

After reviewing the model correctness and potentially comparing its behavior to your other previous version, we can flag our model as ready to be deployed.

*Note: Evaluation can be automated and part of a MLOps step: once you deploy a new Chatbot version with a new prompt, run the evaluation job and benchmark your model behavior vs the previous version.*

In [0]:
client = MlflowClient()
client.set_registered_model_alias(name=MODEL_NAME_FQN, alias="prod", version=model.version)

## Conclusion

Mosaic AI Agent Evaluation makes it easy to evaluate your LLM Models, leveraging custom metrics.

Evaluating your chatbot is key to measure your future version impact, and your Data Intelligence Platform makes it easy leveraging automated Workflow for your MLOps pipelines.

For a production-grade GenAI application, this step should be automated and part as a job, executed everytime the model is changed and benchmarked against previous run to make sure you don't have performance regression.